In [12]:
a = 1 if False else 2

a

2

In [180]:
def solution(a, b):
    import calendar
    lst = ['MON', 'TUE', 'WED', 'THU', 'FRI', 'SAT', 'SUN']
    return lst[calendar.weekday(2016, a, b)]



1


In [175]:
def solution(arr):
    answer = []

    for e in arr:
        if answer[-1:] != [e]:
            answer.append(e)
        else:
            pass


    return answer


solution([1,1,3,3,0,1,1])

[1, 3, 0, 1]

In [176]:
l = []

l[-1:]

[]

In [166]:
def solution(n):
    def primes(n):
        primfac = []
        d = 2
        
        while d*d <= n:
            while (n % d) == 0:
                primfac.append(d)
                n //= d
            d += 1
        if n > 1:
            primfac.append(n)
            
        return len(primfac)

    lst = []
    for i in range(1, n+1):
        if primes(i) == 1:
            lst.append(i)

    return len(lst)


solution(10)

4

In [163]:
def solution(n):
    primfac = []
    d = 2
    
    while d*d <= n:
        while (n % d) == 0:
            primfac.append(d)
            n //= d
        d += 1
    if n > 1:
        primfac.append(n)
        
    return primfac

solution(5)

[5]

In [157]:
def solution(n):
    return '수박'*(n//2) if n%2 == 0 else '수박'*(n//2)+'수'

solution(11)

'수박수박수박수박수박수'

In [155]:
def solution(s):
    return eval(s)

solution('-1234')

-1234

In [153]:
def solution(s, n):
    letter_lower = 'abcdefghijklmnopqrstuvwxyz'
    letter_upper = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    lst = []

    for i in s:
        if i.islower():
            lst.append(letter_lower[(letter_lower.find(i)+n)%26])
        elif i.isupper():
            lst.append(letter_upper[(letter_upper.find(i)+n)%26])
        else:
            lst.append(' ')
    
    return ''.join(lst) 



solution('a B z', 4)

'e F d'

In [150]:
'a'.isupper()

False

In [134]:
def solution(n):
    lst_devisor = []
    i = 1

    while i <= n**(1/2):
        if n % i == 0:
            lst_devisor.append(i)
            lst_devisor.append(n//i)
        i += 1

    return sum(set(lst_devisor))


solution(12)

28

In [131]:
100**(1/2)

10.0

In [ ]:
s = ' hello  wORLd '
l = []

for i in range(len(s)):
    if s[i] == ' ':
        l.append(-1)
    
    if i > 0 and s[i-1] == '':
        s[i]


In [129]:
def solution(s):
    l = s.split(' ')
    lst_element = []

    for e in l:
        for i in range(len(e)):
            if i % 2 == 0:
                lst_element.append(e[i].upper())
            else:
                lst_element.append(e[i].lower())
        lst_element.append(' ')

    return ''.join(lst_element).rstrip()

solution("try hello world")

'TrY HeLlO WoRlD'

In [128]:
def solution(s):

    l = s.split(' ')
    # lst_element = []; lst_sentence = []

    # for e in l:
    #     for i in range(len(e)):
    #         if i % 2 == 0:
    #             lst_element.append(e[i].upper())
    #         else:
    #             lst_element.append(e[i].lower())
    #     lst_element.append(' ')


    a = ''.join([ e[i].upper() if i % 2 == 0 else e[i].lower()for e in l for i in range(len(e))  ])
    # a = ''.join([ e[i].upper() if i % 2 == 0 else e[i].lower() for i in range(len(e)) for e in l ])


    return a

solution("try hello world")

'TrYHeLlOWoRlD'

In [120]:
s = 'abcd'

s[1].upper()

'B'

In [109]:
def solution(n, m):
    from collections import Counter
    from functools import reduce

    answer = []

    def primes(n):
        primfac = []
        d = 2
        while d*d <= n:
            while (n % d) == 0:
                primfac.append(d)
                n //= d
            d += 1
        if n > 1:
            primfac.append(n)
        return primfac

    lst_n = primes(n); lst_m = primes(m)

    list_gcd = list((Counter(lst_n) & Counter(lst_m)).elements())
    list_lcm = list((Counter(lst_n) | Counter(lst_m)).elements())




    return [ reduce(lambda x, y: x*y, list_gcd, 1), reduce(lambda x, y: x*y, list_lcm, 1) ]

solution(2, 5)

[1, 10]

In [97]:
def primes(n):
        primfac = []
        d = 2
        while d*d <= n:
            while (n % d) == 0:
                primfac.append(d)
                n //= d
            d += 1
        if n > 1:
            primfac.append(n)
        return primfac

primes(20)

[2, 2, 5]

In [99]:
10 / 2

5.0

In [95]:
def solution(num):
    n = 0

    while num != 1:
        if num % 2 == 0:
            num /= 2
        else:
            num = num*3 + 1
        
        n += 1
        if n == 500:
            n = -1
            break

    return n


solution(626331)

-1

In [92]:
def solution(x):
    return True if x % sum([ int(i) for i in str(x) ]) == 0 else False

solution(10)

True

In [90]:
s = str(10)
sum([ int(i) for i in s ])



1

In [87]:
def solution(phone_number):
    
    last4 = phone_number[-4:]

    return '*'*(len(phone_number)-4) + last4


solution("01033334444")

'*******4444'

In [85]:
def solution(A,B):
    answer = [[c + d for c, d in zip(a,b)] for a, b in zip(A,B)]

    for a, b in zip(A, B):
        print(a, b)
        for c, d in zip(a, b):
            print(c, d)



    return answer

solution([[1,2],[2,3]], [[3,4],[5,6]])


[1, 2] [3, 4]
1 3
2 4
[2, 3] [5, 6]
2 5
3 6


[[4, 6], [7, 9]]

In [82]:
def solution(arr1, arr2):
    answer = [[]]*len(arr1)

    for i in range(len(arr1)):
        print(i)
        answer[i] = [ arr1[i][j] + arr2[i][j] for j in range(len(arr1[0])) ]
        print(answer[i])



    return answer

solution([[1,2],[2,3]], [[3,4],[5,6]])

[[], []]
0
[4, 6]
1
[7, 9]


[[4, 6], [7, 9]]

In [80]:
l = [[4, 5],[6, 7]]

len(l)




2

In [50]:
def solution(participant, completion):
    answer = ''
    dict_p = {}; dict_c = {}

    for e in participant:
        dict_p[e] = dict_p.get(e, 0) + 1

    for e in completion:
        dict_c[e] = dict_c.get(e, 0) + 1

    for e in dict_p.keys():
        try: 
            if dict_p[e] != dict_c[e]:
                answer = e
                break
        except:
            answer = e
            break

    
    return answer

solution(["marina", "josipa", "nikola", "vinko", "filipa"], ["josipa", "filipa", "marina", "nikola"])


'vinko'

In [42]:
def solution(participant, completion):
    answer = ''
    for idx, t in enumerate(zip(sorted(participant), sorted(completion))):
        print(t)
        if t[0] != t[1]:
            answer = sorted(participant)[idx]
        else:
            answer = sorted(participant)[len(participant)-1]

    return answer

    # return list(set(participant) - set(completion))[0]

solution(["marina", "josipa", "nikola", "vinko", "filipa"], ["josipa", "filipa", "marina", "nikola"])

('filipa', 'filipa')
('josipa', 'josipa')
('marina', 'marina')
('nikola', 'nikola')


'vinko'

In [39]:
def solution(participant, completion):
    for e in completion:
        participant.remove(e)

    return participant[0]

solution(["marina", "josipa", "nikola", "vinko", "filipa"], ["josipa", "filipa", "marina", "nikola"])

'vinko'

In [9]:
def solution(array, commands):

    # map

    return list(map(lambda x:sorted(array[x[0]-1:x[1]])[x[2]-1], commands))

solution([1, 5, 2, 6, 3, 7, 4], [[2, 5, 3], [4, 4, 1], [1, 7, 3]])

[5, 6, 3]

In [6]:
def solution(array, commands):
    answer = []

    for e in commands:
        part = array[e[0]-1:e[1]]
        answer.append(sorted(part)[e[2]-1])

    return answer

solution([1, 5, 2, 6, 3, 7, 4], [[2, 5, 3], [4, 4, 1], [1, 7, 3]])

[5, 6, 3]

In [424]:
def solution(answers):
    answer = []

    first = [1, 2, 3, 4, 5]
    second = [2, 1, 2, 3, 2, 4, 2, 5]
    third = [3, 3, 1, 1, 2, 2, 4, 4, 5, 5]
    dict_ = {}


    # for _ in answers:
    first_lst = [ 1 if i[0] == i[1] else 0 for i in zip(first*(len(answers) // len(first) + 1), answers) ]
    second_lst = [ 1 if i[0] == i[1] else 0 for i in zip(second*(len(answers) // len(second) + 1), answers) ]
    third_lst = [ 1 if i[0] == i[1] else 0 for i in zip(third*(len(answers) // len(third) + 1), answers) ]

    dict_[1] = sum(first_lst)
    dict_[2] = sum(second_lst)
    dict_[3] = sum(third_lst)

    
    return [ i[0] for i in dict_.items() if i[1] == max(dict_.values()) ]

    return dict_
    

    # return first_lst, second_lst, third_lst
    # return max(dict_.values())
    # return [ i[0] for i in sorted(dict_.items(), key=lambda item: item[1], reverse=True) if i[1] == max(dict_.values()) ]


solution([1, 2, 3, 4, 5])

[1]

In [425]:
a = ['a', 'b', 'c']

for idx, v in enumerate(a):
    print(idx, v)


0 a
1 b
2 c


In [375]:
A = [1, 2, 3]
B = [1, 2, 3, 4]
lst = []

for a, b in zip(A, B):
    lst.append(a+b)
    # print(a, b)

lst

[2, 4, 6]

In [370]:
def solution(n, lost, reserve):
    lst_ = [1]*n
    
    for i in lost:
        lst_[i-1] -= 1
    for i in reserve:
        lst_[i-1] += 1

    print(lst_)

    for i in range(n):
        print(i)
        if lst_[i] != 0:
            pass
        else:
            if i == 0:
                if lst_[i+1] == 2:
                    lst_[i] += 1
                    lst_[i+1] -= 1
            elif i == n-1:
                if lst_[i-1] == 2:
                    lst_[i] += 1
                    lst_[i-1] -= 1
            else:
                if lst_[i-1] == 2:
                    lst_[i] += 1
                    lst_[i-1] -= 1
                elif lst_[i+1] == 2:
                    lst_[i] += 1
                    lst_[i+1] -= 1
                else: pass

    return n - lst_.count(0)
    # return lst_


solution(5, [1, 2, 5], [3])

[0, 0, 2, 1, 0]
0
1
2
3
4


3

In [349]:
def solution(N, stages):
    answer = []
    dict_ = { i: stages.count(i) for i in range(1, N+1) }
    print(dict_)
    dict_fail_rate = {}
    acc = len(stages)

    for i in range(1, N+1):
        # print(dict_[i], acc)
        if acc != 0:
            dict_fail_rate[i] = dict_[i] / acc
        else:
            dict_fail_rate[i] = 0
        acc -= dict_[i]


    return [ i[0] for i in sorted(dict_fail_rate.items(), key=lambda item: item[1], reverse=True) ]
    

solution(5, [2, 1, 2, 6, 2, 4, 3, 3])
# solution(3, [1, 1, 1])

{1: 1, 2: 3, 3: 2, 4: 1, 5: 0}


[3, 4, 2, 1, 5]

In [354]:
def solution(N, stages):
    answer = []
    fail = []
    info = [0] * (N + 2)
    for stage in stages:
        info[stage] += 1
    print(info)
    for i in range(N):
        be = sum(info[(i + 1):])
        yet = info[i + 1]
        print('i:', i, '//', be, yet)
        if be == 0:
            fail.append((str(i + 1), 0))
        else:
            fail.append((str(i + 1), yet / be))
        print(fail)
    for item in sorted(fail, key=lambda x: x[1], reverse=True):
        answer.append(int(item[0]))
    return answer

solution(5, [2, 1, 2, 6, 2, 4, 3, 3])

[0, 1, 3, 2, 1, 0, 1]
i: 0 // 8 1
[('1', 0.125)]
i: 1 // 7 3
[('1', 0.125), ('2', 0.42857142857142855)]
i: 2 // 4 2
[('1', 0.125), ('2', 0.42857142857142855), ('3', 0.5)]
i: 3 // 2 1
[('1', 0.125), ('2', 0.42857142857142855), ('3', 0.5), ('4', 0.5)]
i: 4 // 1 0
[('1', 0.125), ('2', 0.42857142857142855), ('3', 0.5), ('4', 0.5), ('5', 0.0)]


[3, 4, 2, 1, 5]

In [337]:
dict_ = {1: 0.125, 2: 0.42857142857142855, 3: 0.5, 4: 0.5, 5: 0.0}
sorted(dict_.items(), key=lambda item: item[1], reverse=True)

[(3, 0.5), (4, 0.5), (2, 0.42857142857142855), (1, 0.125), (5, 0.0)]

In [321]:
lst = (1, 2, 3, 4, 5)
iterator = iter(lst)

iterator

print(next(iterator))
print(next(iterator))




1
2


SyntaxError: 'yield' outside function (185777605.py, line 10)

In [317]:
lst = [1, 2, 3, 4, 5]
def neighborhood(iterable):
        iterator = iter(iterable)
        prev_item = None
        current_item = next(iterator)  # throws StopIteration if empty.
        for next_item in iterator:
            yield (prev_item, current_item, next_item)
            prev_item = current_item
            current_item = next_item
        yield (prev_item, current_item, None)

for i in neighborhood(lst):
    print(i)

(None, 1, 2)
(1, 2, 3)
(2, 3, 4)
(3, 4, 5)
(4, 5, None)


In [301]:
def solution(board, moves):
    lst = []
    answer = 0

    for p in moves:
        i = 0
        while i <= len(board) -1:
            if board[i][p-1] == 0:
                i += 1
            else:
                lst.append(board[i][p-1])
                try:
                    if lst[-1] == lst[-2]:
                        del lst[-2:]
                        answer += 2
                except: pass
                board[i][p-1] = 0
                break

    # def neighborhood(iterable):
    #     iterator = iter(iterable)
    #     prev_item = None
    #     current_item = next(iterator)  # throws StopIteration if empty.
    #     for next_item in iterator:
    #         yield (prev_item, current_item, next_item)
    #         prev_item = current_item
    #         current_item = next_item
    #     yield (prev_item, current_item, None)

    # count = 1
    # while count != 0:
    #     count = 0
    #     for p, c, n in neighborhood(lst):
    #         if c == n:
    #             lst.remove(c); lst.remove(n)
    #             count += 1
                
    return answer


solution([[0,0,0,0,0],[0,0,1,0,3],[0,2,5,0,1],[4,2,4,4,2],[3,5,1,3,1]], [1,5,3,5,1,2,1,4])

before [4]
1 [4]
before [4, 3]
5 [4, 3]
before [4, 3, 1]
3 [4, 3, 1]
before [4, 3, 1, 1]
after [4, 3]
5 [4, 3]
before [4, 3, 3]
after [4]
1 [4]
before [4, 2]
2 [4, 2]
1 [4, 2]
before [4, 2, 4]
4 [4, 2, 4]


4

In [297]:
a = [0, 1, 2, 4, 4]
del a[-2:]
# a[-1] == a[-2]
a

[0, 1, 2]

In [4]:
solution = lambda a, b: (b*2).find(a)
solution('hello', 'ohell')

1

In [11]:
a, b, c = 1, 2
b

ValueError: not enough values to unpack (expected 3, got 2)

In [14]:
a = 0.9
round(a)

1

In [15]:
s = 'abcDEF'
s.upper()

def solution(my_string):
  return ''.join([ item.lower() if item.isupper() else item.upper() for item in my_string ])

solution(s)

s.index('D')

3

In [39]:
def solution(cipher, code):
  return ''.join([item for num, item in enumerate(cipher) if (num+1) % code == 0])

solution('pfqallllabwaoclk', 2)

'fallback'

In [28]:
s = 'pfqallllabwaoclk'
for num, item in enumerate(s):
  print (num, item)

0 p
1 f
2 q
3 a
4 l
5 l
6 l
7 l
8 a
9 b
10 w
11 a
12 o
13 c
14 l
15 k


In [13]:
n = 3693693690000
answer = 0

for item in str(n):
  if int(item) % 3 == 0 and int(item) != 0:
    answer += 1

map(lambda x: x+1, [3, 6, 9])

In [38]:
n = 20
lst = [3, 28, 10, 9, 11, -25, 21, 19]
# for i in sorted(lst):
#   if abs(i - n) == 0:
#     answer = i
#   else:

abs_list = [ abs(item - n) for item in lst ]
min_value = min(abs_list)
min_idx = [ num for num, value in enumerate(abs_list) if value == min_value ]

answer = map(lambda x: lst[x], min_idx)
min(answer)


19

In [48]:
n = 20
lst = [3, 28, 10, 9, 11, -25, 21, 19]

solution=lambda a,n:sorted(a,key=lambda x:(abs(x-n),x))[0]

for item in lst:
  func = lambda x:(abs(item-n),item)
  print(func(item))

solution(lst, n)

(17, 3)
(8, 28)
(10, 10)
(11, 9)
(9, 11)
(45, -25)
(1, 21)
(1, 19)


19

In [51]:
lst = [3, 28, 10, 9, 11, -25, 21, 19]
sorted(lst, key=0)

TypeError: 'int' object is not callable

In [52]:
s = 'hello world'
s.count('h')

1

In [66]:
def solution(bin1, bin2):
  return bin(int(str(bin1), 2) + int(str(bin2), 2))[2:]

solution(10, 11)

'101'

In [63]:
"{0:b}".format(10)

int(str(10), 2)

2

In [69]:
len(str(100))

3

In [81]:
lines = [[0, 5], [3, 9], [1, 10]]

def solution(lines):
  answer = 0; semi_ans = 0
  lst_ext = []
  for item in lines:
    lst_ext.extend(item)

  m, n = max(lst_ext), min(lst_ext)

  for x in range(n, m):
    for item in lines:
      if x + 0.5 > item[0] and x + 0.5 < item[1]:
        semi_ans += 1
        continue
    if semi_ans >= 2:
      answer += 1
    semi_ans = 0
  
  return answer



solution(lines)

8

In [1]:
from itertools import combinations

dots = [[1, 4], [9, 2], [3, 8], [11, 6]]

# comb = []
comb = combinations(dots, 2)
# print(sorted(list(comb)))

for item in list(comb):
  print(item, '*')
  comb_com = sorted([ elm for elm in dots if elm not in item ])
  print(comb_com)

  print((item[1][1]-item[0][1])/(item[1][0]- item[0][0]))
  print((comb_com[1][1]-comb_com[0][1])/(comb_com[1][0]- comb_com[0][0]))

  if (item[1][1]-item[0][1])/(item[1][0]- item[0][0]) == (comb_com[1][1]-comb_com[0][1])/(comb_com[1][0]- comb_com[0][0]):
    answer = 1
    break

answer
  


([1, 4], [9, 2]) *
[[3, 8], [11, 6]]
-0.25
-0.25


1

In [27]:
def solution(n):
    answer = 0
    for _ in range(n):
        print(_, answer)
        answer += 1
        if answer % 3 == 0 or '3' in str(answer):
            answer += 1
    return answer

solution(10)

0 0
1 1
2 2
3 4
4 5
5 7
6 8
7 10
8 11
9 13


14

In [25]:
def solution(n):
    answer = 0
    for _ in range(n):
        print(_, answer)
        answer += 1
        while answer % 3 == 0 or '3' in str(answer):
            answer += 1
    return answer

solution(10)

0 0
1 1
2 2
3 4
4 5
5 7
6 8
7 10
8 11
9 14


16

In [9]:
lst = [0]
dict_ = {}

for i in range(1, 300):
  if '3' in str(i) or i % 3 == 0:
    continue
  else:
    lst.append(i)

for i in range(1, 101):
  dict_[i] = lst[i]

dict_


{1: 1,
 2: 2,
 3: 4,
 4: 5,
 5: 7,
 6: 8,
 7: 10,
 8: 11,
 9: 14,
 10: 16,
 11: 17,
 12: 19,
 13: 20,
 14: 22,
 15: 25,
 16: 26,
 17: 28,
 18: 29,
 19: 40,
 20: 41,
 21: 44,
 22: 46,
 23: 47,
 24: 49,
 25: 50,
 26: 52,
 27: 55,
 28: 56,
 29: 58,
 30: 59,
 31: 61,
 32: 62,
 33: 64,
 34: 65,
 35: 67,
 36: 68,
 37: 70,
 38: 71,
 39: 74,
 40: 76,
 41: 77,
 42: 79,
 43: 80,
 44: 82,
 45: 85,
 46: 86,
 47: 88,
 48: 89,
 49: 91,
 50: 92,
 51: 94,
 52: 95,
 53: 97,
 54: 98,
 55: 100,
 56: 101,
 57: 104,
 58: 106,
 59: 107,
 60: 109,
 61: 110,
 62: 112,
 63: 115,
 64: 116,
 65: 118,
 66: 119,
 67: 121,
 68: 122,
 69: 124,
 70: 125,
 71: 127,
 72: 128,
 73: 140,
 74: 142,
 75: 145,
 76: 146,
 77: 148,
 78: 149,
 79: 151,
 80: 152,
 81: 154,
 82: 155,
 83: 157,
 84: 158,
 85: 160,
 86: 161,
 87: 164,
 88: 166,
 89: 167,
 90: 169,
 91: 170,
 92: 172,
 93: 175,
 94: 176,
 95: 178,
 96: 179,
 97: 181,
 98: 182,
 99: 184,
 100: 185}

In [5]:
num_dict = {'0': 0}
num_cum = 0

def jump_(n):
  if '3' in str(n) or n % 3 == 0:
    return jump_(n+1)
  else: 
    return n

for i in range(1, 101):
  if '3' in str(num_cum + 1) or (num_cum +  1) % 3 == 0:
    num_dict[str(i)] = jump_(i)
    num_cum = jump_(i)
  else:
    num_dict[str(i)] = num_cum + 1
    num_cum += 1
  
  
  

num_dict

{'0': 0,
 '1': 1,
 '2': 2,
 '3': 4,
 '4': 5,
 '5': 5,
 '6': 7,
 '7': 8,
 '8': 8,
 '9': 10,
 '10': 11,
 '11': 11,
 '12': 14,
 '13': 14,
 '14': 14,
 '15': 16,
 '16': 17,
 '17': 17,
 '18': 19,
 '19': 20,
 '20': 20,
 '21': 22,
 '22': 22,
 '23': 25,
 '24': 26,
 '25': 25,
 '26': 26,
 '27': 28,
 '28': 29,
 '29': 29,
 '30': 40,
 '31': 41,
 '32': 40,
 '33': 41,
 '34': 40,
 '35': 41,
 '36': 40,
 '37': 41,
 '38': 40,
 '39': 41,
 '40': 40,
 '41': 41,
 '42': 44,
 '43': 44,
 '44': 44,
 '45': 46,
 '46': 47,
 '47': 47,
 '48': 49,
 '49': 50,
 '50': 50,
 '51': 52,
 '52': 52,
 '53': 55,
 '54': 56,
 '55': 55,
 '56': 56,
 '57': 58,
 '58': 59,
 '59': 59,
 '60': 61,
 '61': 62,
 '62': 62,
 '63': 64,
 '64': 65,
 '65': 65,
 '66': 67,
 '67': 68,
 '68': 68,
 '69': 70,
 '70': 71,
 '71': 71,
 '72': 74,
 '73': 74,
 '74': 74,
 '75': 76,
 '76': 77,
 '77': 77,
 '78': 79,
 '79': 80,
 '80': 80,
 '81': 82,
 '82': 82,
 '83': 85,
 '84': 86,
 '85': 85,
 '86': 86,
 '87': 88,
 '88': 89,
 '89': 89,
 '90': 91,
 '91': 92,
 '92': 

In [2]:
num_lst = []
num_cum = 0

def jump_(n):
  if '3' in str(n) or n % 3 == 0:
    return jump_(n+1)
  else: 
    return n

for i in range(1, 101):
  if '3' in str(i) or i % 3 == 0:
    num_lst.append(jump_(i))
    num_cum = jump_(i)
  else:
    num_lst.append(num_cum + 1)
    num_cum += 1

num_lst

[1,
 2,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 14,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 25,
 25,
 26,
 27,
 28,
 29,
 30,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 40,
 41,
 42,
 44,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 55,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 74,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 85,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101]

In [3]:
num_dict = {}
num_cum = 1

def jump_(n):
  if '3' in str(n) or n % 3 == 0:
    return jump_(n+1)
  else: 
    return n

for i in range(1, 101):
  if '3' in str(i) or i % 3 == 0:
    num_dict[str(i)] = jump_(i)
    num_cum = jump_(i)
  else:
    num_dict[str(i)] = num_cum + 1
    num_cum += 1
  
  
  

num_dict


{'1': 1,
 '2': 2,
 '3': 4,
 '4': 5,
 '5': 6,
 '6': 7,
 '7': 8,
 '8': 9,
 '9': 10,
 '10': 11,
 '11': 12,
 '12': 14,
 '13': 14,
 '14': 15,
 '15': 16,
 '16': 17,
 '17': 18,
 '18': 19,
 '19': 20,
 '20': 21,
 '21': 22,
 '22': 23,
 '23': 25,
 '24': 25,
 '25': 26,
 '26': 27,
 '27': 28,
 '28': 29,
 '29': 30,
 '30': 40,
 '31': 40,
 '32': 40,
 '33': 40,
 '34': 40,
 '35': 40,
 '36': 40,
 '37': 40,
 '38': 40,
 '39': 40,
 '40': 41,
 '41': 42,
 '42': 44,
 '43': 44,
 '44': 45,
 '45': 46,
 '46': 47,
 '47': 48,
 '48': 49,
 '49': 50,
 '50': 51,
 '51': 52,
 '52': 53,
 '53': 55,
 '54': 55,
 '55': 56,
 '56': 57,
 '57': 58,
 '58': 59,
 '59': 60,
 '60': 61,
 '61': 62,
 '62': 63,
 '63': 64,
 '64': 65,
 '65': 66,
 '66': 67,
 '67': 68,
 '68': 69,
 '69': 70,
 '70': 71,
 '71': 72,
 '72': 74,
 '73': 74,
 '74': 75,
 '75': 76,
 '76': 77,
 '77': 78,
 '78': 79,
 '79': 80,
 '80': 81,
 '81': 82,
 '82': 83,
 '83': 85,
 '84': 85,
 '85': 86,
 '86': 87,
 '87': 88,
 '88': 89,
 '89': 90,
 '90': 91,
 '91': 92,
 '92': 93,
 '93'

In [2]:
def solution(n):
    answer = 0
    num_dict = {}

    def jump_(num):
        if '3' in str(num) or num % 3 == 0:
            num_dict[str(num)] = jump_(num+1)
            # return jump_(num+1)
        else:
            print(num)
            num_dict[str(num)] = num
            print(num_dict)
            return num_dict[str(num)]
    

    for i in range(1, 101):
        print(i, jump_(i))
    
    # return num_dict(n)

solution(1)


1
{'1': 1}
1 1
2
{'1': 1, '2': 2}
2 2
4
{'1': 1, '2': 2, '4': 4}
3 None
4
{'1': 1, '2': 2, '4': 4, '3': 4}
4 4
5
{'1': 1, '2': 2, '4': 4, '3': 4, '5': 5}
5 5
7
{'1': 1, '2': 2, '4': 4, '3': 4, '5': 5, '7': 7}
6 None
7
{'1': 1, '2': 2, '4': 4, '3': 4, '5': 5, '7': 7, '6': 7}
7 7
8
{'1': 1, '2': 2, '4': 4, '3': 4, '5': 5, '7': 7, '6': 7, '8': 8}
8 8
10
{'1': 1, '2': 2, '4': 4, '3': 4, '5': 5, '7': 7, '6': 7, '8': 8, '10': 10}
9 None
10
{'1': 1, '2': 2, '4': 4, '3': 4, '5': 5, '7': 7, '6': 7, '8': 8, '10': 10, '9': 10}
10 10
11
{'1': 1, '2': 2, '4': 4, '3': 4, '5': 5, '7': 7, '6': 7, '8': 8, '10': 10, '9': 10, '11': 11}
11 11
14
{'1': 1, '2': 2, '4': 4, '3': 4, '5': 5, '7': 7, '6': 7, '8': 8, '10': 10, '9': 10, '11': 11, '14': 14}
12 None
14
{'1': 1, '2': 2, '4': 4, '3': 4, '5': 5, '7': 7, '6': 7, '8': 8, '10': 10, '9': 10, '11': 11, '14': 14, '13': 14, '12': None}
13 None
14
{'1': 1, '2': 2, '4': 4, '3': 4, '5': 5, '7': 7, '6': 7, '8': 8, '10': 10, '9': 10, '11': 11, '14': 14, '13': 14, 

In [145]:
num_dict = {'1': 1}
num_dict['1']

1

In [131]:
a_dict = [1, 2, 3]

a_dict[1] = 1

a_dict

[1, 1, 3]

In [118]:
range(0, 1) in range(0, 2)

False

In [95]:
lst = [1, 2, 3]
lst2 = map(lambda x: x+1, lst)
list(lst2)

[2, 3, 4]

In [47]:
spell = ["s", "o", "m", "d"]
dic = ["moos", "dzx", "smm", "sunmmo", "som"]

def solution(spell, dic):
  from itertools import permutations
  answer = 0
  spell_lst = [ ''.join(item) for item in (permutations(spell, len(spell))) ]
  print(list(spell_lst))
  return 1 if set(dic) & set(spell_lst) else 2

solution(spell, dic)

['somd', 'sodm', 'smod', 'smdo', 'sdom', 'sdmo', 'osmd', 'osdm', 'omsd', 'omds', 'odsm', 'odms', 'msod', 'msdo', 'mosd', 'mods', 'mdso', 'mdos', 'dsom', 'dsmo', 'dosm', 'doms', 'dmso', 'dmos']


2

In [43]:
from itertools import permutations
spell_lst = [ ''.join(item) for item in (permutations(spell, 3)) ]
set(dic).intersection(set(spell_lst))

set()

In [50]:
def solution(sides):
  answer = 0

  int_max = sides[0] + sides[1] - 1
  int_min = max(sides) - min(sides) + 1
  print(len(range(int_min, int_max)) + 1 )

  return answer

solution([11, 7])

13


0

In [70]:
import numpy as np

board = [[1, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 0]]

arr_board = np.array(board)

for i in range(len(arr_board)):
  for j in range(len(arr_board)):
    if arr_board[i, j] == 1:
      arr_board[0 if i < 0 or i > len(arr_board) - 1 else i - 1, 0 if j < 0 or j > len(arr_board) -1 else j - 1] = 2

arr_board

array([[1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 2, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 0, 2]])

In [68]:
len(arr_board)

5

In [1]:
def solution(my_string):
  int_lst = []

  for item in my_string:
    item_ext = []
    while item.isdigit():
      item_ext.append(item)
    int_lst.append(item_ext)
    
  return int_lst

my_string = "aAb1B2cC34oOp"
solution(my_string)

KeyboardInterrupt: 

In [8]:
from itertools import cycle

lst = list("aAb1B2cC34oOp")
n = 0

while n < len(lst):
  for elem, next_elem in zip(lst, lst[1:]+[lst[0]]):
    print(elem, next_elem)
    n += 1

# print(zip(lst, lst[1:]+[lst[0]]))

a A
A b
b 1
1 B
B 2
2 c
c C
C 3
3 4
4 o
o O
O p
p a


In [42]:
def solution(my_string):
  s = ''.join([ i if i.isdigit() else ' ' for i in my_string ])
  return sum([ int(i) for i in s.split()])

my_string = "aAb1B2cC34oOp"
solution(my_string)

37

In [36]:
def solution(my_string):
  int_lst = []

  for item in my_string:
    if not item.isdigit():
      int_lst.append('/')
    else:
      int_lst.append(item)
  
  return sum([ 0 if item == '' else int(item) for item in ''.join(int_lst).split('/') ])

my_string = "aAb1B2cC34oOp"
solution(my_string)

37

In [195]:
polynomial =  "3"

def solution(polynomial):

  lst = polynomial.split(' + ')
  b = sum([ int(i) for i in lst if i.isdigit() ])
  x_lst = [ i.rstrip('x') for i in lst if not i.isdigit() ]
  a = sum([ 1 if i == '' else int(i) for i in x_lst ])
  print(x_lst, a, b)

  if a == 0:
    return str(b)
  elif b == 0:
    repl = '{}x'.format(a)[0:2].replace('1x', 'x')
    return repl + '{}x'.format(a)[2:]
  else:
    repl = '{}x + {}'.format(a, b)[0:2].replace('1x', 'x')
    return repl + '{}x + {}'.format(a, b)[2:]
      

solution(polynomial)

[] 0 3


'3'

In [169]:
polynomial =  "x + x"

def solution(polynomial):

  if polynomial == "":
    return "0"

  else:

    try: 
      lst = polynomial.split(' + ')
      b = sum([ int(i) for i in lst if i.isdigit() ])
      x_lst = [ i.rstrip('x') for i in lst if not i.isdigit() ]
      a = sum([ 1 if i == '' else int(i) for i in x_lst ])
      print(x_lst, a, b)

      if a == 0:
        return b
      elif b == 0:
        repl = '{}x'.format(a)[0:2].replace('1x', 'x')
        return repl + '{}x'.format(a)[2:]
      else:
        repl = '{}x + {}'.format(a, b)[0:2].replace('1x', 'x')
        return repl + '{}x + {}'.format(a, b)[2:]
      
    except:
      return "0"



solution(polynomial)

'0'

In [201]:
def solution(s):
    lst = s.split(' '); lst_int = []
    
    for i in range(len(lst)):
      try:
        lst_int.append(int(lst[i]))
      except:
        lst_int.append(-int(lst[i-1]))

    return sum(lst_int)

solution("1 2 Z 3")

4

In [211]:
def solution(n):
    primfac = []
    d = 2
    while d*d <= n:
        while (n % d) == 0:
            primfac.append(d)
            n //= d
        d += 1
    if n > 1:
        primfac.append(n)
    return [ int(i) for i in set(primfac) ]


solution(420)

[2, 3, 5, 7]

In [218]:
def solution(my_string):
    return sum([ int(i) if i.isdigit() else 0 for i in my_string ])

solution("aAb1B2cC34oOp")

10

In [224]:
def solution(n):
    fac = 1; i = 1
    while fac <= n:
        fac *= i
        i += 1
    
    return i - 2

solution(7)

3

In [285]:
def solution(num_list, n):
    answer = [[]]
    # lst = ['*']*3
    answer = zip(*(iter(num_list),)*n)
    return list(answer)

solution([100, 95, 2, 4, 5, 6, 18, 33, 948]	, 3)

[(100, 95, 2), (4, 5, 6), (18, 33, 948)]

In [246]:

iter([100, 95, 2, 4, 5, 6, 18, 33, 948])

TypeError: 'list_iterator' object is not callable

In [252]:
a = [1, 2, 3, 4]
b = iter(a)

print(b.__next__())
print(b.__next__())
print(b.__next__())
print(b.__next__())

1
2
3
4


In [253]:
a = [1, 2, 3, 4]

*(iter(a))

SyntaxError: can't use starred expression here (753764916.py, line 3)

In [287]:
def solution(dot):
    if dot[0] > 0 and dot[1] > 0:
        answer = 1
    elif dot[0] < 0 and dot[1] > 0:
        answer = 2
    elif dot[0] < 0 and dot[1] < 0:
        answer = 3
    else:
        answer = 4
    return answer

solution([-7, 9])

2

In [296]:
def solution(age):
    str_ = 'abcdefghij'
    return ''.join( str_[int(item)] for item in str(age) )

solution(56)

'fg'

In [294]:
str_ = 'abcdefghij'
list(str_)

str(56)[0]

'5'

In [297]:
def solution(n):
    answer = 0
    while n > 0:
        if n % 2 == 0:
            answer += n
        n -= 1
    return answer

solution(10)

30

In [3]:
n = 20

a = sum(filter(lambda x: x % 2 == 0, [1, 2, 3, 4]))
a


6

In [11]:
def solution(n, k):

    sheep = n*12000
    service = n // 10
    cost_beverage = (k - service)*2000 if k >= service else 0

    return sheep + cost_beverage

solution(10, 0)

120000

In [12]:
def solution(n, k):
    return 12000 * n + 2000 * (k - n // 10)

solution(10, 0)

118000

In [13]:
def solution(angle):
    a, b, c, d = (0, 90), 90, (90, 180), 180
    return a

solution(40)

(0, 90)

In [20]:
def solution(my_string, n):
    answer = ''
    return ''.join(map(lambda x: x*3, my_string))

solution('domyBB', 4)

'dddooommmyyyBBBBBB'

In [25]:
def solution(num_list):
    return [ sum(map(lambda x: x % 2 == 0, num_list)), len(num_list) - sum(map(lambda x: x % 2 == 0, num_list)) ]

solution([1, 2, 3, 4, 5])

[2, 3]

In [27]:
for i in range(1, 5):
    print('*'*(i))

*
**
***
****


In [10]:
def solution(my_string):
    return ''.join(list(reversed(my_string)))

solution('jaaron')


'noraaj'

In [9]:
s = 'jaron'

list(reversed(s))

['n', 'o', 'r', 'a', 'j']

In [11]:
def solution(num_list):
    answer = []
    return num_list[::-1]

solution([1, 0, 1, 1, 1, 3, 5])

[5, 3, 1, 1, 1, 0, 1]

In [14]:
import numpy as np
np.average([89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

94.0

In [39]:
def solution(array):

    set_ = set(array)
    dict_ = { k: array.count(k) for k in set_ }
    max_value = max(dict_.values())
    answer = [k for k, v in dict_.items() if v == max_value]

    return -1 if len(answer) != 1 else answer[0]
    
solution([1, 1, 2, 2])

-1

In [45]:
def solution(array):
    return sorted(array)[int((len(array)+1)/2 - 1)]

solution([1, 2, 7, 10, 11])

[2.0]

In [47]:
import numpy as np
list(np.array([1, 2, 3, 4])*2)

[2, 4, 6, 8]

In [49]:
list((0.25).as_integer_ratio())


[1, 4]

In [95]:
# from math import lcm
def solution(numer1, denom1, numer2, denom2):
    from fractions import Fraction
    import math

    lcm_ = denom1*denom2 / math.gcd(denom1, denom2)
    a = numer1 * (lcm_/denom1)
    c = numer2 * (lcm_/denom2)
    answer = Fraction(int(a+c), int(lcm_))
    return [ answer.numerator, answer.denominator ]

solution(1, 2, 1, 2)

[1, 1]

In [99]:
from fractions import Fraction

def solution(denum1, num1, denum2, num2):
    print(Fraction(denum1, num1), Fraction(denum2, num2))
    answer = Fraction(denum1, num1) + Fraction(denum2, num2)
    print(answer)
    return [answer.numerator, answer.denominator]

solution(9, 2, 1, 3)

9/2 1/3
29/6


[29, 6]

In [119]:
def solution(array, height):
    array.append(height)
    return sorted(array, reverse=True).index(height)

solution([149, 180, 192, 170], 167)

1


3

In [137]:
a = [[1, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 0, 0]]
a[-1][-1]

0

In [165]:
def solution(board):
    bomb = 0
    dummy = [(-1, -1), (0, -1), (1, -1), (-1, 0), (0, 0), (1, 0), (-1, 1), (0, 1), (1, 1)]

    for i in range(len(board)):
      for j in range(len(board)):
        for k in range(len(dummy)):
          try:
            print(i, j, '**', i+dummy[k][0], j+dummy[k][1], '***', board[i+dummy[k][0]][j+dummy[k][1]])
            if board[i+dummy[k][0]][j+dummy[k][1]] != 1 or i+dummy[k][0] == -1 or j+dummy[k][1] == -1:
              pass
            else:
              print('bomb')
              bomb += 1
              break
          except:
            continue
        # continue

    return len(board)**2 - bomb

solution([[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 1, 1, 0], [0, 0, 0, 0, 0]])

0 0 ** -1 -1 *** 0
0 0 ** 0 -1 *** 0
0 0 ** 1 -1 *** 0
0 0 ** -1 0 *** 0
0 0 ** 0 0 *** 0
0 0 ** 1 0 *** 0
0 0 ** -1 1 *** 0
0 0 ** 0 1 *** 0
0 0 ** 1 1 *** 0
0 1 ** -1 0 *** 0
0 1 ** 0 0 *** 0
0 1 ** 1 0 *** 0
0 1 ** -1 1 *** 0
0 1 ** 0 1 *** 0
0 1 ** 1 1 *** 0
0 1 ** -1 2 *** 0
0 1 ** 0 2 *** 0
0 1 ** 1 2 *** 0
0 2 ** -1 1 *** 0
0 2 ** 0 1 *** 0
0 2 ** 1 1 *** 0
0 2 ** -1 2 *** 0
0 2 ** 0 2 *** 0
0 2 ** 1 2 *** 0
0 2 ** -1 3 *** 0
0 2 ** 0 3 *** 0
0 2 ** 1 3 *** 0
0 3 ** -1 2 *** 0
0 3 ** 0 2 *** 0
0 3 ** 1 2 *** 0
0 3 ** -1 3 *** 0
0 3 ** 0 3 *** 0
0 3 ** 1 3 *** 0
0 3 ** -1 4 *** 0
0 3 ** 0 4 *** 0
0 3 ** 1 4 *** 0
0 4 ** -1 3 *** 0
0 4 ** 0 3 *** 0
0 4 ** 1 3 *** 0
0 4 ** -1 4 *** 0
0 4 ** 0 4 *** 0
0 4 ** 1 4 *** 0
1 0 ** 0 -1 *** 0
1 0 ** 1 -1 *** 0
1 0 ** 2 -1 *** 0
1 0 ** 0 0 *** 0
1 0 ** 1 0 *** 0
1 0 ** 2 0 *** 0
1 0 ** 0 1 *** 0
1 0 ** 1 1 *** 0
1 0 ** 2 1 *** 0
1 1 ** 0 0 *** 0
1 1 ** 1 0 *** 0
1 1 ** 2 0 *** 0
1 1 ** 0 1 *** 0
1 1 ** 1 1 *** 0
1 1 ** 2 1 *** 0
1 1 ** 0 2 

13

In [124]:
dummy = [(-1, -1), (0, -1), (1, -1), (-1, 0), (0, 0), (1, 0), (-1, 1), (0, 1), (1, 1)]
dummy[0]

(-1, -1)

In [214]:
def solution(today, terms, privacies):
    answer = []
    days_lst = [ (int(today[0:4]) - int(item[0:4]))*12*28 + (int(today[5:7]) - int(item[5:7]))*28 + (int(today[8:10]) - int(item[8:10]) + 1) for item in privacies ]
    terms_dict = { item[0]: int(item.split()[1]) for item in terms}
    pv_term = [ item[-1] for item in privacies]
    # for d, t in zip(days_lst, pv_term):
    #     print(d, t)
    #     if d > int(terms_dict[t])*28:
    #         print(d, int(terms_dict[t])*28, '파기')
    # return terms_dict
    return [ n + 1 for n, item in enumerate(zip(days_lst, pv_term)) if item[0] > int(terms_dict[item[1]])*28 ]

solution("2022.05.19", ["A 6", "B 12", "C 3"], ["2021.05.02 A", "2021.07.01 B", "2022.02.19 C", "2022.02.20 C"])



[1, 3]

In [205]:
a = [1, 2]
b = [3, 4]

for n, x in enumerate(zip(a, b)):
  print(n, x[1])

0 3
1 4


In [212]:
a = "B 12"
a.split()[1]

'12'

In [226]:
def solution(t, p):
    answer = 0
    for i in range(len(t)-len(p)+1):
      print(int(t[i:i+len(p)]))
      if int(t[i:i+len(p)]) <= int(p):
        answer += 1
    # return sum([ 1 for i in range(len(t)-len(p)) if int(t[i:i+len(p)]) <= int(p) ])
    return answer

solution("10203"	, "15")

10
2
20
3


3

In [231]:
def solution(s):
  answer = []
  for i in range(len(s)):
    # print(s.count(s[0:i+1]))
    if s[0:i+1].count(s[i]) == 1:
      answer.append(-1)
    else:
      d = s[i]; g = 1
      while d != s[i-g]:
        g += 1
      answer.append(g)
  return answer


solution("foobar")

[-1, -1, 1, -1, -1, -1]

In [ ]:
def solution(s):
  answer = 0; num = 1
  while True:
    for i in range(1, len(s)):
      if s[i] != s[0]:
        answer += 1
        s = s[i+1:]
      else:
        num += 1

  return answer


solution("aaabbaccccabba")

In [ ]:
answer = 0

def solution(s):
  global answer
  
  def main(s):
    print(s)
    global answer
    if s[0] != s[1]:
      # s.pop(0); s.pop(0)
      s = s[2:]
      print(s)
      answer += 1
      print(s)
    else:
      for i in range(4, len(s), 2):
        if s[:i].count(s[0]) == len(s[:i])/2:
          s = s[i:]
          print
          answer += 1
  
  while len(s) > 1:
    main(s)
  
  answer += 1

  return answer


solution("aaabbaccccabba")

In [309]:
def solution(s):
  answer = 0

  while len(s) > 3:
    if s[0] != s[1]:
      s = s[2:]
      answer += 1
      print(s, answer)
    else:
      for i in range(4, len(s), 2):
        if s[:i].count(s[0]) == len(s[:i])/2:
          s = s[i:]
          print(s)
          answer += 1
  
  if len(s) == 3 and s[0] != s[1]:
    answer += 1
  else:
    pass

  return answer if len(s) == 0 else answer + 1
  # return answer


solution("aaabbaccccabba")

ccabba
ba


3

In [ ]:
def solution(s):
  answer = 0

  for i in range(len(s)):
    if s[i] != s[i+1]:
      answer += 1
      s = s[i+2:]
    else:
      for j in range(i+2, len(s)):
        if 



  return answer

solution("aaabbaccccabba")

In [242]:
a = [ 1, 2, 3, 4, 5 ]

a.pop((0, 1))

a


# for i in range(0, len(a), 2):
#   print(i)

TypeError: 'tuple' object cannot be interpreted as an integer

In [310]:
a = 'ssssffff'
list(reversed(a))

['f', 'f', 'f', 'f', 's', 's', 's', 's']

In [319]:
def solution(number, limit, power):
    def q_fac(n):
        return len([ i for i in range(1, int((n+1)/2)+1) if n % i == 0 ]) + 1

    lst = [ q_fac(i) for i in range(1, number+1)]
    # print(lst)
    limited_lst = [ power if i > limit else i for i in lst ]

    return sum(limited_lst) - 1

solution(10, 3, 2)

21

In [370]:
def solution(number, limit, power):
    import functools
    def primes(n):
        primfac = []
        d = 2
        while d*d <= n:
            while (n % d) == 0:
                primfac.append(d)
                n //= d
            d += 1
        if n > 1:
            primfac.append(n)
        return primfac

    def q_fac(n):
        set_ = set(primes(n))
        dict_ = { i: primes(n).count(i) + 1 for i in set_ }
        lst_ = [ dict_[i] for i in dict_.keys() ]
        # print(dict_)
        return functools.reduce(lambda x, y: x*y, lst_)

    lst = [ q_fac(i) if i != 1 else 1 for i in range(1, number+1)]
    limited_lst = [ power if i > limit else i for i in lst ]

    return sum(limited_lst)

solution(10, 3, 2)


21

In [367]:
import functools

def primes(n):
  primfac = []
  d = 2
  while d*d <= n:
      while (n % d) == 0:
          primfac.append(d)
          n //= d
      d += 1
  if n > 1:
      primfac.append(n)
  return primfac


def q_fac(n):
  set_ = set(primes(n))
  dict_ = { i: primes(n).count(i) + 1 for i in set_ }
  lst = [ dict_[i] for i in dict_.keys() ]
  print(dict_)
  return functools.reduce(lambda x, y: x*y, lst)

q_fac(2)
  

{2: 2}


2

In [358]:
a = [1, 2, 3]

dict2_ = { 3: 2, 4: 5}

[ dict2_[i] for i in dict2_.keys() ]

[2, 5]

In [325]:
def q_fac(n):
        return len([ i for i in range(1, int((n+1)/2)+1) if n % i == 0 ]) + 1

q_fac(10)

4

In [330]:
import functools

q_fac2 = functools.reduce(lambda x, y: dict_[x]*dict_[y], dict_)

q_fac2

9

In [390]:
def solution(k, m, score):
    answer = 0
    sorted_ = sorted(score, reverse=True)
    lst = []
    for i in range(0, len(score), m):
      try:
        lst.append([ sorted_[j+i] for j in range(m) ])
        print(lst)
      except:
        pass

    print([ i[m-1]*m for i in lst])

    return sum([ i[m-1]*m for i in lst])


solution(4, 3, [4, 1, 2, 2, 4, 4, 4, 4, 1, 2, 4, 2, 10])

[[10, 4, 4]]
[[10, 4, 4], [4, 4, 4]]
[[10, 4, 4], [4, 4, 4], [4, 2, 2]]
[[10, 4, 4], [4, 4, 4], [4, 2, 2], [2, 2, 1]]
[12, 12, 6, 3]


33

In [391]:
lst = [1, 2, 3, 4, 5, 6, 7, 8]

lst[2::3]

[3, 6]

In [401]:
def solution(food):
    lst_ = []

    for i in range(1, len(food)):
      for j in range(food[i]//2):
        lst_.append(i)

    return ''.join( str(i) for i in lst_ + [0] + list(reversed(lst_)) )

solution([1, 3, 4, 6])

'1223330333221'

In [5]:
def solution(ingredient):
    answer = 0
    hamburger_mat = [1, 2, 3, 1]
    count_ = len(ingredient)
    hamburger = []
    ingre_left = []
    
    while count_ > 0:
      print(count_)
      for i in hamburger_mat:
        count_ -= 1
        for j in ingredient:
          print(j)
          if j == i:
            hamburger.append(j)
            ingredient.remove(j)
            print(ingredient)
            break
          else:
            ingre_left.append(j)

    return hamburger


solution([2, 1, 1, 2, 3, 1, 2, 3, 1])

9
2
1
[2, 1, 2, 3, 1, 2, 3, 1]
2
[1, 2, 3, 1, 2, 3, 1]
1
2
3
[1, 2, 1, 2, 3, 1]
1
[2, 1, 2, 3, 1]
5
2
1
[2, 2, 3, 1]
2
[2, 3, 1]
2
3
[2, 1]
2
1
[2]
1
2
2
[]


[1, 2, 3, 1, 1, 2, 3, 1, 2]

In [ ]:
def solution(babbling):

  word = ["aya", "ye", "woo", "ma"]

  for i in babbling:
    while True:
      if i[0:2] == 'ye':
        i = i[]



  return answer

In [9]:
a = ' abc / d '

b = a.strip()

len(a)


9

In [16]:
def solution(a, b, n):
    answer = 0

    while n >= a:
      add_coke, bottle_left = divmod(n, a)
      answer += add_coke*b
      n = add_coke*b + bottle_left

    return answer

solution(3, 1, 20)

9

In [14]:
a, b = divmod(20, 3)
a, b

(6, 2)

In [23]:
def solution(number):
    from itertools import combinations

    lst = [ sum(i) for i in list(combinations(number, 3)) ]

    return lst.count(0)


solution([-3, -2, -1, 0, 1, 2, 3])

5

In [122]:
def solution(X, Y):
    answer = ''

    dict_X = { i: 0 for i in '0123456789' }
    dict_Y = { i: 0 for i in '0123456789' }

    for i in X:
      dict_X[i] += 1
    for j in Y:
      dict_Y[j] += 1
    
    for i in range(9, -1, -1):
      i = str(i)
      iter_num = min(dict_X[i], dict_Y[i])
      answer = ''.join([answer, i*iter_num])

    if len(set(answer)) == 1 and '0' in answer:
      answer = '0'
      
    return '-1' if answer == '' else answer


solution("100", "203045")

'0'

In [134]:
def solution(n):
    remainder = 0
    i = 2
    while remainder != 1:
      remainder = n%i
      i += 1

    return i-1

solution(100)

3

In [132]:
def solution(n):
    answer = 0
    for i in range(2, n):
        if n % i == 1:
            return i
    # return answer

solution(100)

3

In [135]:
def solution(sizes):
    max_w = 0; max_h = 0

    for i in sizes:
      if sorted(i)[0] > max_w:
        max_w = sorted(i)[0]
      if sorted(i)[1] > max_h:
        max_h = sorted(i)[1]

    return max_w*max_h

solution([[60, 50], [30, 70], [60, 30], [80, 40]])

4000

In [146]:
sum([[60, 50], [30, 70], [60, 30], [80, 40]], [])
sum([1, 2, 3, 4], 5)

15

In [153]:
sum(set([10, 20, 30]) - set([20, 40]))

40

In [156]:
def solution(s):
    dict_ = {'zero': '0', 'one': '1', 'two': '2', 'three': '3', 'four': '4', 'five': '5', 'six': '6', 'seven': '7', 'eight': '8', 'nine': '9'}
    
    for i in dict_:
      s = s.replace(i, dict_[i])
    
    return s

solution("one4seveneight")

'1478'

In [176]:
from string import ascii_lowercase

def solution(s, skip, index):
    result = ''

    a_to_z = set(ascii_lowercase)
    a_to_z -= set(skip)
    print(a_to_z)
    a_to_z = sorted(a_to_z)
    print(a_to_z)
    l = len(a_to_z)
    print(l)

    dic_alpha = {alpha:idx for idx, alpha in enumerate(a_to_z)}
    print(dic_alpha)

    for i in s:
        result += a_to_z[(dic_alpha[i] + index) % l]

    return result

solution("z", "abcdefghij", 20)

{'y', 'k', 'x', 's', 'w', 'u', 'l', 'r', 't', 'p', 'z', 'n', 'o', 'v', 'q', 'm'}
['k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
16
{'k': 0, 'l': 1, 'm': 2, 'n': 3, 'o': 4, 'p': 5, 'q': 6, 'r': 7, 's': 8, 't': 9, 'u': 10, 'v': 11, 'w': 12, 'x': 13, 'y': 14, 'z': 15}


'n'

In [211]:
def solution(numbers, hand):
    answer = ''
    numbers = [ 11 if i == 0 else i for i in numbers ]
    l_former = 10; r_former = 12

    for i in numbers:
        if i in [1, 4, 7]:
            answer += 'L'
            l_former = i
        elif i in [3, 6, 9]:
            answer += 'R'
            r_former = i
        else:
            # print(i, l_former, r_former)

            d_l_i_v = abs((l_former-1)//3 - (i-1)//3)
            d_l_i_h = 0 if abs((l_former - i)%3) == 0 else 1
            d_l_i = d_l_i_v + d_l_i_h
            # print(d_l_i, d_l_i_v, d_l_i_h)

            d_r_i_v = abs((r_former-1)//3 - (i-1)//3)
            d_r_i_h = 0 if abs((r_former - i)%3) == 0 else 1
            d_r_i = d_r_i_v + d_r_i_h
            # print(d_r_i, d_r_i_v, d_r_i_h)

            if d_l_i < d_r_i:
                l_former = i
                answer += 'L'
            elif d_l_i > d_r_i:
                r_former = i
                answer += 'R'
            else:
                if hand == 'left':
                    l_former = i
                else:
                    r_former = i
                answer += hand[0].upper()
            

    return answer


solution([1, 2, 3, 4, 5, 6, 7, 8, 9, 0], "right")

'LLRLLRLLRL'

In [285]:
def solution(board, moves):
    import pandas as pd
    lst = []

    df = pd.DataFrame(board)
    # print(df)

    for p in moves:
        i = 0
        while i <= len(df.index) -1:
            if df.iloc[i][p-1] == 0:
                i += 1
            else:
                lst.append(df.iloc[i][p-1])
                df.at[i, p-1] = 0
                break
            # print(df)

        # print(p, lst)
    
    num_lst = len(lst)

    def neighborhood(iterable):
        iterator = iter(iterable)
        prev_item = None
        current_item = next(iterator)  # throws StopIteration if empty.
        for next_item in iterator:
            yield (prev_item, current_item, next_item)
            prev_item = current_item
            current_item = next_item
        yield (prev_item, current_item, None)

    count = 1
    while count != 0:
        count = 0
        for p, c, n in neighborhood(lst):
            if c == n:
                lst.remove(c); lst.remove(n)
                count += 1
                
    return num_lst - len(lst)



solution([[0,0,0,0,0],[0,0,1,0,3],[0,2,5,0,1],[4,2,4,4,2],[3,5,1,3,1]], [1,5,3,5,1,2,1,4])

   0  1  2  3  4
0  0  0  0  0  0
1  0  0  1  0  3
2  0  2  5  0  1
3  4  2  4  4  2
4  3  5  1  3  1
1 [4]
5 [4, 3]
3 [4, 3, 1]
5 [4, 3, 1, 1]
1 [4, 3, 1, 1, 3]
2 [4, 3, 1, 1, 3, 2]
1 [4, 3, 1, 1, 3, 2]
4 [4, 3, 1, 1, 3, 2, 4]


4

In [272]:
lst = [4, 3, 1, 1, 3, 2, 4]
answer = 0

while True:
    for i in range(0, len(lst) - 1):
        print(i, lst[i], lst[i+1])
        if lst[i] == lst[i+1]:
            lst.remove(lst[i]); lst.remove(lst[i])
            answer += 2
            print(lst)

answer

0 4 3
1 3 1
2 1 1
[4, 3, 3, 2, 4]
3 2 4


IndexError: list index out of range

In [284]:
lst = [4, 3, 1, 1, 3, 2, 4]

def neighborhood(iterable):
    iterator = iter(iterable)
    prev_item = None
    current_item = next(iterator)  # throws StopIteration if empty.
    for next_item in iterator:
        yield (prev_item, current_item, next_item)
        prev_item = current_item
        current_item = next_item
    yield (prev_item, current_item, None)

count = 1
while count != 0:
    count = 0
    for p, c, n in neighborhood(lst):
        if c == n:
            lst.remove(c); lst.remove(n)
            count += 1
            print(lst)




[4, 3, 3, 2, 4]
[4, 2, 4]


In [282]:
lst = [4, 3, 1, 1, 3, 2, 4]
len(set(lst))
# len(lst)

4

In [248]:
import pandas as pd

df = pd.DataFrame([[0,0,0,0,0],[0,0,1,0,3],[0,2,5,0,1],[4,2,4,4,2],[3,5,1,3,1], [3,5,1,3,1]])
print(df)

df.at[3, 2] = '*'

len(df.index)

   0  1  2  3  4
0  0  0  0  0  0
1  0  0  1  0  3
2  0  2  5  0  1
3  4  2  4  4  2
4  3  5  1  3  1
5  3  5  1  3  1


6

In [259]:
import functools, itertools
n = 0
# lst = functools.filter(lambda x, y: '' if x == y, [4, 3, 1, 1, 3, 2, 4])
lst = filter(lambda x, y: x == y, [4, 3, 1, 1, 3, 2, 4])

list(lst)


TypeError: <lambda>() missing 1 required positional argument: 'y'

In [207]:
numbers = [1, 3, 4, 5, 8, 2, 1, 4, 5, 0, 5]
numbers = [ 11 if i == 0 else i for i in numbers]
numbers

[1, 3, 4, 5, 8, 2, 1, 4, 5, 11, 5]

In [199]:
a = 1; b = 2
a if a == 2 else b


2

In [178]:
a = 'str'
a[0].upper()

'S'

In [177]:
def solution(a, b):
    import numpy as np
    return np.inner(a, b)


solution([1,2,3,4], [-3,-1,0,2])

3

In [171]:
chr(116)

't'

In [159]:
set([1, 2, 3])

{1, 2, 3}

In [158]:
for i, v in enumerate(['a', 'b', 'c']):
  print(i, v)

0 a
1 b
2 c


In [130]:
import math
math.sqrt(4000)

63.245553203367585

In [119]:
a = '00'
len(set(a))

1

In [94]:
a = 'abc'
b = '123'

c = ''.join([a, b])
c

'abc123'

In [61]:
a = 'aaabbb'

a.count('a')

3

In [26]:
a = '12345'
a.replace('1', '')


'2345'